# [모듈 5.1] 모델 빌딩 파이프라인 개발 (SageMaker Model Building Pipeline 모든 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 30분** 소요 됩니다.

- 0. SageMaker Model Building Pipeline 개요
- 1. 파이프라인 변수 및 환경 설정
- 2. 파이프라인 스텝 단계 정의

    - (1) 전처리 스텝 스텝 정의    
    - (2) 모델 학습을 위한 학습 스텝 정의 
    - (3) 모델 평가 스텝
    - (4) 모델 등록 스텝
    - (5) HPO 단계    
    - (6) 최고 모델 등록 스텝    
    - (7) 모델 승인 상태 변경 람다 스텝    
    - (8) 배포할 세이지 메이커 모델 스텝 생성
    - (9) 모델 앤드 포인트 배포를 위한 람다 스텝 생성    
    - (5) 세이지 메이커 모델 생성 스텝 생성    
    - (6) HPO 스텝
    - (7) 조건 스텝
    - (10) 조건 스텝
    
- 3. 모델 빌딩 파이프라인 정의 및 실행
- 4. Pipleline 캐싱 및 파라미터 이용한 실행
- 5. 정리 작업
    
---

# 0.SageMaker Model Building Pipeline 개요

Amazon SageMaker 모델 구축 파이프라인은 직접 SageMaker 통합을 활용하는 머신 러닝 파이프라인을 구축하기 위한 도구입니다. 이러한 통합으로 인해 많은 단계 생성 및 관리를 처리하는 도구를 사용하여 파이프라인을 생성하고 오케스트레이션용 SageMaker Projects를 설정할 수 있습니다. SageMaker 파이프라인은 다른 파이프라인에 비해 다음과 같은 이점을 제공합니다

- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)

# 1. 파이프라인 변수 및 환경 설정



In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

%store -r 

## (1) 모델 빌딩 파이프라인 변수 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 프로세싱 스텝을 위한 인스턴스 타입 및 인스턴스 수
    - 데이터 전처리 스텝 및 실시간 앤드 포인트 스텝에 사용 됨.
- 훈련 스텝을 위한 인스턴스 타입 및 인스턴스 수    
- 모델 평가를 통해 나온 validation:roc-auc 에 대한 분기 조건 값
- 원본 데이터 세트에 대한 S3 주소
    - 데이터 전처리 스텝에서 사용 됩니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

model_eval_threshold = ParameterFloat(
    name="model2eval2threshold",
    default_value=0.85
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


## 캐싱 정의

- 참고: 캐싱 파이프라인 단계:  [Caching Pipeline Steps](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

In [3]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, 
                           expire_after="7d")


# 2. 파이프라인 스텝 단계 정의

## (1) 전처리 스텝 단계 정의
- input_data_uri 입력 데이타를 대상으로 전처리를 수행 합니다.

크게 아래와 같은 순서로 정의 합니다.
- 프로세싱 오브젝트 정의 (SKLearnProcessor)
- 프로세싱 스텝 정의
    - 일력 데이터 세트
        - source: S3 경로 (input_data_uri)
        - destination: 도커 컨테이너의 내부 폴더 위치
    - 출력 위치
        - 훈련 전처리 데이터 결과 위치
        - 테스트 전처리 데이터 결과 위치
    - 프로세싱 코드
    - 프로세싱 코드에 넘길 인자 


In [4]:
from sagemaker.sklearn.processing import SKLearnProcessor

split_rate = 0.2
framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)
print("input_data: \n", input_data)

input_data: 
 s3://sagemaker-us-east-1-028703291518/sagemaker-pipeline-step-by-step-phase02/input


In [5]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    
step_process = ProcessingStep(
    name="FraudProcess",
    processor=sklearn_processor,
    inputs=[
#         ProcessingInput(source=input_data_uri,destination='/opt/ml/processing/input'),
        ProcessingInput(source=input_data, destination='/opt/ml/processing/input'),        
         ],
    outputs=[ProcessingOutput(output_name="train",
                              source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name="test",
                              source='/opt/ml/processing/output/test')],
    job_arguments=["--split_rate", f"{split_rate}"],        
    code= 'src/preprocessing.py',
    cache_config = cache_config, # 캐시 정의
)


## (2) 모델 학습을 위한 학습단계 정의 

학습 스텝을 정의하기 위해서는 크게 아래와 같은 과정이 있습니다.
- XGBoost Estimator 정의
- 학습 스텝 정의
    - 아래와 같은 중요한 인자가 필요 합니다.
        - Estimator (위에서 정의한 것 사용)
        - 훈련을 위한 입력 데이터 위치


### 기본 훈련 변수 및 하이퍼파라미터 설정

In [6]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = 'fraud2train'
estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'

base_hyperparameters = {
       "scale_pos_weight" : "29",        
        "max_depth": "6",
        "alpha" : "0", 
        "eta": "0.3",
        "min_child_weight": "1",
        "objective": "binary:logistic",
        "num_round": "100",
}


In [7]:
xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = base_hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

훈련의 입력이 이전 전처리의 결과가 제공됩니다.
- `step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri`

In [8]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },
    cache_config = cache_config, # 캐시 정의    
)

## (3) 모델 평가 단계

### SKLearnProcessor 의 기본 도커 컨테이너 지정
ScriptProcessor 의 기본 도커 컨테이너로 Scikit-learn를 기본 이미지를 사용함. 
- 사용자가 정의한 도커 컨테이너도 사용할 수 있습니다.

In [9]:
from sagemaker.processing import ScriptProcessor

script_eval = SKLearnProcessor(
                             framework_version= "0.23-1",
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=1,
                             base_job_name="script-fraud-scratch-eval",
                                    )


모델 평가 지표를 jons 파일에 저장을 하고 property로 등록을 합니다.

In [10]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

step_eval = ProcessingStep(
    name="FraudEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
        destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="src/evaluation.py",
    cache_config = cache_config, # 캐시 정의    
  property_files=[evaluation_report], 
)

## (4) 모델 등록 스텝

### 모델 그룹 생성
- 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 안에 모델 패키지의 모델 버전 등록시에 삽입함

- 참고
    - 모델 그룹 릭스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)
    - 모델 지표 등록: [Model Quality Metrics](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)

In [11]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}
response = sm_client.list_model_package_groups(NameContains=model_package_group_name)
if len(response['ModelPackageGroupSummaryList']) == 0:
    print("No model group exists")
    print("Create model group")    
    
    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))    
else:
    print(f"{model_package_group_name} exitss")

sagemaker-pipeline-step-by-step-phase02 exitss


In [12]:
from sagemaker.workflow.step_collections import RegisterModel

from sagemaker.model_metrics import MetricsSource, ModelMetrics 

# 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 에 모데 버전 등록시에 삽입함
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)


step_register = RegisterModel(
    name= "FraudcRegisterhModel",
    estimator=xgb_train,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

## (5) HPO 스텝
- 하이퍼 파라미터 튜닝 스텝을 정의 합니다.

In [13]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}


objective_metric_name = "validation:auc"

tuner = HyperparameterTuner(
    xgb_train, objective_metric_name, hyperparameter_ranges, 
    max_jobs=5,
    max_parallel_jobs=5,
)

from sagemaker.workflow.steps import TuningStep
    
step_tuning = TuningStep(
    name = "FraudTuning",
    tuner = tuner,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },    
    cache_config = cache_config, # 캐시 정의        
)

## (6) 최고 모델 등록 스텝

### 최고의 모델 생성 및 등록

하이퍼파라미터 튜닝 작업을 성공적으로 완료한 후TuningStep의 교육 작업에 의해 생성된 모델 아티팩트에서 SageMaker 모델을 생성하거나 모델 레지스트리에 모델을 등록할 수 있습니다.


TuningStep 클래스의 get_top_model_s3_uri 메서드를 사용하여 최고 성능의 모델 버전의 모델 아티팩트를 가져옵니다.

- `model_bucket_key` 는 튜닝 스텝을 통해서 생성된 훈련 잡의 위치 입니다.

In [14]:
model_bucket_key = estimator_output_path.split('//')[1]
print("model_bucket_key: ", model_bucket_key)
print("model_package_group_name: ", model_package_group_name)

model_bucket_key:  sagemaker-us-east-1-028703291518/fraud2train/training_jobs
model_package_group_name:  sagemaker-pipeline-step-by-step-phase02


In [15]:
from sagemaker.workflow.step_collections import RegisterModel


step_register_best = RegisterModel(
    name="RegisterBestFraudModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)

## (7) 모델 승인 상태 변경 람다 스텝
- 모델 레지스트리에서 해당 모델 패키지 그룹을 조회하고, 가장 최신 버전의 모델에 대해서 '모델 승인 상태 변경' 을 합니다.

#### 참고: 
`step_approve_lambda.add_depends_on([step_register_best])`시에 에러 발생
- 에러 발생 원인은 명시적으로 `RegisterModel` 은 의존성을 허용하지 않아서 나는 것으로 추정이 됨.
- 그래서 의존 스텝을 step_tuning 으로 지정 함.

```
ValueError: Invalid input step name: RegisterModel(steps=[_RegisterModelStep(name='test', step_type=<StepTypeEnum.REGISTER_MODEL: 'RegisterModel'>, depends_on=None)])
```

#### [에러] 
아래와 같은 데러가 발생시에 `0.0.Setup-Environment.ipynb` 의 정책 추가 부분을 진행 해주세요.
```
ClientError: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::0287032915XX:assumed-role/AmazonSageMaker-ExecutionRole-20210827T141955/SageMaker is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::0287032915XX:role/lambda-deployment-role
```

In [16]:
from src.iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role")
print("lambda_role: \n", lambda_role)

lambda_role: 
 arn:aws:iam::028703291518:role/lambda-deployment-role


In [17]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

import time 

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "sagemaker-lambda-step-approve-model-deployment-" + current_time

print("function_name: \n", function_name)

function_name: 
 sagemaker-lambda-step-approve-model-deployment-08-27-08-24-45


In [18]:
# Lambda helper class can be used to create the Lambda function
func_approve_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_change_model_approval.py",
    handler="iam_change_model_approval.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_approve_lambda = LambdaStep(
    name="LambdaApproveModelStep",
    lambda_func=func_approve_model,
    inputs={
        "model_package_group_name" : model_package_group_name,
        "ModelApprovalStatus": "Approved",
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)
step_approve_lambda.add_depends_on([step_tuning])
# step_approve_lambda.add_depends_on([step_register_best])

## (8) 배포할 세이지 메이커 모델 스텝 생성
- 위의 람다 스텝에서 "모델 승인 상태" 를 변경한 모델에 대하여 '모델 레지스트리'에서 저장된 도커 컨테이너 이미지, 모델 아티펙트의 위치를 가져 옵니다.
- 이후에 이 두개의 인자를 가지고 세이지 메이커 모델을 생성 합니다.

In [19]:
import boto3
sm_client = boto3.client('sagemaker')

# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)

ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)
response = sm_client.describe_model_package(ModelPackageName=ModelPackageArn)
image_uri_approved = response["InferenceSpecification"]["Containers"][0]["Image"]
ModelDataUrl_approved = response["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
print("image_uri_approved: ", image_uri_approved)
print("ModelDataUrl_approved: ", ModelDataUrl_approved)

IndexError: list index out of range

In [ ]:
from sagemaker.model import Model
    
model = Model(
    image_uri= image_uri_approved,
    model_data= ModelDataUrl_approved,    
    sagemaker_session=sagemaker_session,
    role=role,
)

In [ ]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_best_model = CreateModelStep(
    name="CreateFraudhModel",
    model=model,
    inputs=inputs,
)
step_create_best_model.add_depends_on([step_approve_lambda]) # step_approve_lambda 완료 후 실행 함.

## (9) 모델 앤드 포인트 배포를 위한 람다 스텝 생성
- 람다 함수는 입력으로 세이지 메이커 모델, 앤드 포인트 컨피그 및 앤드 포인트 이름을 받아서, 앤드포인트를 생성 함.


In [ ]:
# model_name = project_prefix + "-lambda-model" + current_time
endpoint_config_name = "lambda-deploy-endpoint-config-" + current_time
endpoint_name = "lambda-deploy-endpoint-" + current_time

function_name = "sagemaker-lambda-step-endpoint-deploy-" + current_time

# print("model_name: \n", model_name)
print("endpoint_config_name: \n", endpoint_config_name)
print("endpoint_config_name: \n", len(endpoint_config_name))
print("endpoint_name: \n", endpoint_name)
print("function_name: \n", function_name)




In [ ]:
# Lambda helper class can be used to create the Lambda function
func_deploy_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_create_endpoint.py",
    handler="iam_create_endpoint.lambda_handler",
    timeout = 900, # 디폴트는 120초 임. 10분으로 연장
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaDeployStep",
    lambda_func=func_deploy_model,
    inputs={
        "model_name": step_create_best_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

In [ ]:
# 다음 노트북의 추론을 위해서 저장
all_adv_pipeline_endpoint_name = endpoint_name
%store all_adv_pipeline_endpoint_name

## (10) 조건 스텝

In [ ]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    # right=8.0
    right = model_eval_threshold
)

step_cond = ConditionStep(
    name="FruadMetricCond",
    conditions=[cond_lte],
    if_steps=[step_tuning],        
    else_steps=[step_register], 
)

# 3.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 20분 정도 소요 됩니다.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'sagemaker-pipeline-phase2-step-by-step'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        training_instance_count,                
        input_data,
        model_eval_threshold,
        model_approval_status,        
    ],

    
  steps=[step_process, step_train, step_eval, step_cond, step_register_best, step_approve_lambda, 
         step_create_best_model, step_deploy_lambda],
)



In [ ]:
all_adv_pipeline_name = pipeline_name
%store all_adv_pipeline_name

In [ ]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 


In [ ]:
pipeline.upsert(role_arn=role)

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [ ]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [ ]:
execution.describe()

In [ ]:
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [ ]:
execution.list_steps()

# 4. Pipeline 캐싱 및 파라미터 이용한 실행
- 캐싱은 2021년 7월 현재 Training, Processing, Transform 의 Step에 적용이 되어 있습니다.
- 상세 사항은 여기를 확인하세요. -->  [캐싱 파이프라인 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)


In [ ]:
is_cache = True

In [ ]:
%%time 

from IPython.display import display as dp
import time

if is_cache:
    execution = pipeline.start(
        parameters=dict(
            model2eval2threshold=0.8,
        )
    )    
    
    # execution = pipeline.start()
    time.sleep(10)
    dp(execution.list_steps())    
    execution.wait()


In [ ]:
if is_cache:
    dp(execution.list_steps())

# 5. 정리 작업

#### 아티펙트 경로 추출

In [ ]:
def get_proc_artifact(execution, client, kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()

    proc_arn = response[-1]['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
    #proc_arn = response[-1]['Metadata']
    # print("proc_arn: ", proc_arn)
    proc_job_name = proc_arn.split('/')[-1]
    print("proc_job_name: ", proc_job_name)
    
    response = client.describe_processing_job(ProcessingJobName = proc_job_name)
    test_preprocessed_file = response['ProcessingOutputConfig']['Outputs'][kind]['S3Output']['S3Uri'] # index 1: test 파일    
    print("test_preprocessed_file: \n ", test_preprocessed_file)
    
    return test_preprocessed_file

import boto3
client = boto3.client("sagemaker")

test_preproc_dir_artifact = get_proc_artifact(execution, client, kind=1 )
train_preproc_dir_artifact = get_proc_artifact(execution, client, kind=0 )

#print("test_preproc__dir_artifact: ", test_preproc_dir_artifact)



## 변수 저장

In [ ]:
%store test_preproc_dir_artifact
%store train_preproc_dir_artifact